# New notebook to keep dev clean

In [69]:
import os
import sys
import praw
from praw import Reddit

class WallpaperScraper:
    reddit_oath = {
        'client_id': 'pIxpnAoiGfwE-g',
        'client_secret': 'Ueccpv4dJegXUYbmAIdwxevxjDs',
        'user_agent': 'wallpapers'
    }
    
    imgur_oath = {
    'client_id': '29b27fc1363aa92',
    'client_secret': 'ef58b651b764dfb487809e64a2d62982e1a6392e'
    }
    
    imgur_headers = {
      'Authorization': f'Client-ID {imgur_oath["client_id"]}'
    }
    
#     sort_options = {'controversial', 'gilded', 'hot', 'new', 'rising', 'top'}
#     time_filter = {'all', 'day', 'hour', 'month', 'week', 'year'}
    
    def __init__(self, img_dir, sort='top', time_filter='week'):
        """
        Initialize a wallpaper subreddit scraper instance with feed options.
        """
        
        self._img_dir = img_dir
        self._mk_img_dir()
        
        self._wallpapers = Reddit(**self.reddit_oath).subreddit('wallpapers')
        if sort in {'controversial', 'top'}:
            self._wallpapers = getattr(self._wallpapers, sort)(time_filter=time_filter)
        else:
            self._wallpapers = getattr(self._wallpapers, sort)()
        
    def _mk_img_dir(self):
        """
        Checks and initializes a directory for image files to be downloaded.
        """
        if not os.path.exists(self._img_dir):
            os.mkdir(self._img_dir)
        if not os.path.exists(os.path.join(self._img_dir, 'buffer')):
            os.mkdir(os.path.join(self._img_dir, 'buffer'))
            
    def _img_url_extract(self, img_sub):
        """
        Extract and return the url hyperlink of only PRAW's submission-type objects.
        """
        if isinstance(img_sub, praw.models.reddit.submission.Submission):
            return img_sub.url
        return None

In [83]:
test = WallpaperScraper('wallpapers', 'controversial', 'year')

In [84]:
for idx, img_sub in enumerate(test._wallpapers):
    if isinstance(img_sub, praw.models.reddit.submission.Submission):
        img_sub.url.split('.')[-1]
        file_ext = img_sub.url.split('.')[-1]
        print(img_sub.url, file_ext, img_sub.thumbnail)
    #     with open(f'tmp/tmp{idx}.{file_ext}', 'wb') as tmp:
    #         r = requests.get(img_sub.url, stream=True)
    #         if r.ok:
    #             tmp.write(r.content)
    #     file_ext = img_sub.thumbnail.split('.')[-1]
    #     with open(f'tmp/tmp{idx}_.{file_ext}', 'wb') as tmp:
    #         r = requests.get(img_sub.thumbnail, stream=True)
    #         if r.ok:
    #             tmp.write(r.content)
    
    if idx == 10:
        break

https://imgur.com/a/22D4xOJ com/a/22D4xOJ https://b.thumbs.redditmedia.com/SSWGbJdT06dgxC7DPRUBMy2FjzV_JfwovqVVISHdgjI.jpg
https://i.redd.it/o1vmuo8vend21.jpg jpg https://b.thumbs.redditmedia.com/e8B7NrIZktnovKk-92O2KTZrXm9_TvSG8oJz3CgYjiI.jpg
https://i.redd.it/urs703bti4f31.jpg jpg https://b.thumbs.redditmedia.com/gTMLYkASEJ-AUwROYXGDx2pjAWz1NcBJCftzDTpXtkU.jpg
https://i.redd.it/w3j6kry5mvu21.jpg jpg https://a.thumbs.redditmedia.com/_nnn6idTvszlVdR9X9YT26zRkGsucCLSb-_jLCdbC74.jpg
https://i.redd.it/dc3m37tcbdj31.jpg jpg https://b.thumbs.redditmedia.com/KEKE9kIJ9-qK-GduhhggVD1W9T1G5GFrKm6oF0pwprw.jpg
https://i.redd.it/bmp9m1mrzyg31.jpg jpg https://b.thumbs.redditmedia.com/-lUkWa9M9BmewRGV2HzaG6-xBtzPWXjdwIGJnPIjWMs.jpg
https://i.redd.it/5fcdmvjfzmy21.jpg jpg https://b.thumbs.redditmedia.com/Y_Zo2ch29bXxsKh1n8CGJORAytnHnYT32rbaGbWu8cg.jpg
https://i.redd.it/75fvuwxirjy21.jpg jpg https://b.thumbs.redditmedia.com/KbcC0ccUjB77SMZ7A5Y-gh3BBG2Evolx8_dlwnrlsGM.jpg
https://i.redd.it/hne6x6uo84h3

# Need to develop a class - image generator

- Scrape only scrapable images
    - Predictable scrapes: e.g. imgur/reddit
        - Reddit API provides a thumbnail!! - Great for speed-up of image processing and IO
    - Unpredictable scrapes: Other sites/reddit comments
- Develop a pre-buffer
    - Image pops are quick, but buffer can buffer in the background
    - Store image in buffer folder
    > 1 - URL Buffer
    
    > 2 - Image Buffer

In [82]:
def _img_url_extract(img_sub):
    """
    Extract and return the url hyperlink of only PRAW's submission-type objects.
    Include the link to a thumbnail if it exists
    """
    if isinstance(img_sub, praw.models.reddit.submission.Submission):
        return (img_sub.url, img_sub.thumbnail)
    return None

In [ ]:
def _valid_url(url):
    

In [ ]:
def imgur_gallery_generator(gallery_url):
    assert gallery_url
    
# https://imgur.com/a/WepxW
# https://imgur.com/a/rSZlZ